In [5]:
import requests
from bs4 import BeautifulSoup
import pickle
from time import sleep

In [20]:
class Scraper:
    def __init__(self,city,base_url):
        self.base_url = base_url
        self.attraction_url = self.CreateAttractionUrl()
        self.BASE_HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
        "accept-language": "en-US,en;q=0.9",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        'Referer': 'https://www.tripadvisor.com'
    }
        self.Attraction_Headers = self.BASE_HEADERS.copy()
        self.Attraction_Headers.update({'Referer':self.base_url})
        self.description = ''
        self.popular_experiences = []
        self.top_attractions = []
        self.city_text = ''
        self.attraction_text = ''
        self.city = city
        print(f'Start {self.city}')
        
    def CreateAttractionUrl(self):
        splits = self.base_url.split('-')
        return f'https://www.tripadvisor.com/Attractions-{splits[1]}-Activities-{splits[2]}.html'
    def crawl(self):
        city = requests.get(self.base_url, headers=self.BASE_HEADERS)
        attraction = requests.get(self.attraction_url,headers = self.Attraction_Headers)
        try_time = 0
        
        while(city.status_code != 200):
            print(f'Try {try_time} to crawl city data')
            sleep(10)
            city = requests.get(self.base_url, headers=self.BASE_HEADERS)
            try_time += 1
        
        city_bs= BeautifulSoup(city.text)
        self.city_text = city.text
        self.description = city_bs.find('div', attrs={'data-test-target': 'geo-description'}).text
        try_time = 0
        
        while(attraction.status_code != 200):
            print(f'Try {try_time} to crawl attraction data')
            sleep(10)
            attraction = requests.get(self.attraction_url,headers = self.Attraction_Headers)
            try_time += 1
        self.attraction_text = attraction.text
        attraction_bs = BeautifulSoup(attraction.text)
        self.popular_experiences = [x.text for x in attraction_bs.find_all('span', class_='biGQs _P vvmrG')]
        
    def save(self):
# Load the data from the pickle file
        file_path = f"{self.city}.pkl"
        with open(file_path, "wb") as f:
            pickle.dump(self,f)


In [37]:
import pandas as pd
cities = pd.read_csv('Top50_USA.csv')

In [41]:
cities = cities.loc[46:]

In [42]:
cities

,Rank,Location,LocationId,URL,State
46,47,Indianapolis,37209,https://www.tripadvisor.com/Tourism-g37209-Ind...,IN
47,48,Detroit,42139,https://www.tripadvisor.com/Tourism-g42139-Det...,MI
48,49,Sacramento,32999,https://www.tripadvisor.com/Tourism-g32999-Sac...,CA
49,50,Oakland,32810,https://www.tripadvisor.com/Tourism-g32810-Oak...,CA


In [43]:
for index,row in cities.iterrows():
    city_obj = Scraper(row['Location'],row['URL'])
    city_obj.crawl()
    city_obj.save()
    sleep(5)

Start Indianapolis
Start Detroit
Start Sacramento
Start Oakland


In [32]:
city_obj.base_url

'https://www.tripadvisor.com/Tourism-g48990-Burlington_North_Carolina-Vacations.html'